In [ ]:
"""
🌙 Moon Dev's LLM Trading Agent
Handles all LLM-based trading decisions
"""

import anthropic
import openai
import os
import pandas as pd
import json
from termcolor import colored, cprint
from dotenv import load_dotenv
from datetime import datetime, timedelta
import time

# Local imports
from src import config
from src import nice_funcs as n
from src.data.ohlcv_collector import collect_all_tokens

# Keep only these prompts
TRADING_PROMPT = """
You are Moon Dev's AI Trading Assistant 🌙

Analyze the provided market data and strategy signals (if available) to make a trading decision.

Market Data Criteria:
1. Price action relative to MA20 and MA40
2. RSI levels and trend
3. Volume patterns
4. Recent price movements

{strategy_context}

Respond in this exact format:
1. First line must be one of: BUY, SELL, or NOTHING (in caps)
2. Then explain your reasoning, including:
   - Technical analysis
   - Strategy signals analysis (if available)
   - Risk factors
   - Market conditions
   - Confidence level (as a percentage, e.g. 75%)

Remember: 
- Moon Dev always prioritizes risk management! 🛡️
- Never trade USDC or SOL directly
- Consider both technical and strategy signals
"""

ALLOCATION_PROMPT = """
You are Moon Dev's Portfolio Allocation Assistant 🌙

Given the total portfolio size and trading recommendations, allocate capital efficiently.
Consider:
1. Position sizing based on confidence levels
2. Risk distribution
3. Keep cash buffer as specified
4. Maximum allocation per position

Format your response as a Python dictionary:
{
    "token_address": allocated_amount,  # In USD
    ...
    "USDC_ADDRESS": remaining_cash  # Always use USDC_ADDRESS for cash
}

Remember:
- Total allocations must not exceed total_size
- Higher confidence should get larger allocations
- Never allocate more than {MAX_POSITION_PERCENTAGE}% to a single position
- Keep at least {CASH_PERCENTAGE}% in USDC as safety buffer
- Only allocate to BUY recommendations
- Cash must be stored as USDC using USDC_ADDRESS: {USDC_ADDRESS}
"""


# Load environment variables
load_dotenv()


class TradingAgent:
    def __init__(self):
        # self.client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_KEY"))
        self.client = openai.OpenAI(
            base_url="http://localhost:11434/v1", api_key="ollama"
        )
        self.recommendations_df = pd.DataFrame(
            columns=["token", "action", "confidence", "reasoning"]
        )
        print("🤖 Moon Dev's LLM Trading Agent initialized!")

    def analyze_market_data(self, token, market_data):
        """Analyze market data using Claude"""
        try:
            # Skip analysis for excluded tokens
            if token in config.EXCLUDED_TOKENS:
                print(f"⚠️ Skipping analysis for excluded token: {token}")
                return None

            # Prepare strategy context
            strategy_context = ""
            if "strategy_signals" in market_data:
                strategy_context = f"""
                Strategy Signals Available:
                {json.dumps(market_data['strategy_signals'], indent=2)}
                """
            else:
                strategy_context = "No strategy signals available."

            message = self.client.messages.create(
                model=config.AI_MODEL,
                max_tokens=config.AI_MAX_TOKENS,
                temperature=config.AI_TEMPERATURE,
                messages=[
                    {
                        "role": "user",
                        "content": f"{TRADING_PROMPT.format(strategy_context=strategy_context)}\n\nMarket Data to Analyze:\n{market_data}",
                    }
                ],
            )

            # Parse the response - handle both string and list responses
            response = message.content
            if isinstance(response, list):
                # Extract text from TextBlock objects if present
                response = "\n".join(
                    [
                        item.text if hasattr(item, "text") else str(item)
                        for item in response
                    ]
                )

            lines = response.split("\n")
            action = lines[0].strip() if lines else "NOTHING"

            # Extract confidence from the response (assuming it's mentioned as a percentage)
            confidence = 0
            for line in lines:
                if "confidence" in line.lower():
                    # Extract number from string like "Confidence: 75%"
                    try:
                        confidence = int("".join(filter(str.isdigit, line)))
                    except:
                        confidence = 50  # Default if not found

            # Add to recommendations DataFrame with proper reasoning
            reasoning = (
                "\n".join(lines[1:])
                if len(lines) > 1
                else "No detailed reasoning provided"
            )
            self.recommendations_df = pd.concat(
                [
                    self.recommendations_df,
                    pd.DataFrame(
                        [
                            {
                                "token": token,
                                "action": action,
                                "confidence": confidence,
                                "reasoning": reasoning,
                            }
                        ]
                    ),
                ],
                ignore_index=True,
            )

            print(f"🎯 Moon Dev's AI Analysis Complete for {token[:4]}!")
            return response

        except Exception as e:
            print(f"❌ Error in AI analysis: {str(e)}")
            # Still add to DataFrame even on error, but mark as NOTHING with 0 confidence
            self.recommendations_df = pd.concat(
                [
                    self.recommendations_df,
                    pd.DataFrame(
                        [
                            {
                                "token": token,
                                "action": "NOTHING",
                                "confidence": 0,
                                "reasoning": f"Error during analysis: {str(e)}",
                            }
                        ]
                    ),
                ],
                ignore_index=True,
            )
            return None

    def allocate_portfolio(self):
        """Get AI-recommended portfolio allocation"""
        try:
            cprint("\n💰 Calculating optimal portfolio allocation...", "cyan")
            max_position_size = config.usd_size * (config.MAX_POSITION_PERCENTAGE / 100)
            cprint(
                f"🎯 Maximum position size: ${max_position_size:.2f} ({config.MAX_POSITION_PERCENTAGE}% of ${config.usd_size:.2f})",
                "cyan",
            )

            # Get allocation from AI
            message = self.client.chat.completions.create(
                model=config.AI_MODEL,
                max_tokens=config.AI_MAX_TOKENS,
                temperature=config.AI_TEMPERATURE,
                messages=[
                    {
                    "role": "user",
                    "content": f"""You are Moon Dev's Portfolio Allocation AI 🌙

                    Given:
                    - Total portfolio size: ${config.usd_size}
                    - Maximum position size: ${max_position_size} ({config.MAX_POSITION_PERCENTAGE}% of total)
                    - Minimum cash (USDC) buffer: {config.CASH_PERCENTAGE}%
                    - Available tokens: {config.MONITORED_TOKENS}
                    - USDC Address: {config.USDC_ADDRESS}

                    Provide a portfolio allocation that:
                    1. Never exceeds max position size per token
                    2. Maintains minimum cash buffer
                    3. Returns allocation as a JSON object with token addresses as keys and USD amounts as values
                    4. Uses exact USDC address: {config.USDC_ADDRESS} for cash allocation

                    Example format:
                    {{
                        "token_address": amount_in_usd,
                        "{config.USDC_ADDRESS}": remaining_cash_amount  # Use exact USDC address
                    }}""",
                                        }
                                    ],
                                )

            # Parse the response
            allocations = self.parse_allocation_response(str(message.content))
            if not allocations:
                return None

            # Fix USDC address if needed
            if "USDC_ADDRESS" in allocations:
                amount = allocations.pop("USDC_ADDRESS")
                allocations[config.USDC_ADDRESS] = amount

            # Validate allocation totals
            total_allocated = sum(allocations.values())
            if total_allocated > config.usd_size:
                cprint(
                    f"❌ Total allocation ${total_allocated:.2f} exceeds portfolio size ${config.usd_size:.2f}",
                    "red",
                )
                return None

            # Print allocations
            cprint("\n📊 Portfolio Allocation:", "green")
            for token, amount in allocations.items():
                token_display = "USDC" if token == config.USDC_ADDRESS else token
                cprint(f"  • {token_display}: ${amount:.2f}", "green")

            return allocations

        except Exception as e:
            cprint(f"❌ Error in portfolio allocation: {str(e)}", "red")
            return None

    def execute_allocations(self, allocation_dict):
        """Execute the allocations using AI entry for each position"""
        try:
            print("\n🚀 Moon Dev executing portfolio allocations...")

            for token, amount in allocation_dict.items():
                # Skip USDC and other excluded tokens
                if token in config.EXCLUDED_TOKENS:
                    print(f"💵 Keeping ${amount:.2f} in {token}")
                    continue

                print(f"\n🎯 Processing allocation for {token}...")

                try:
                    # Get current position value
                    current_position = n.get_token_balance_usd(token)
                    target_allocation = amount

                    print(f"🎯 Target allocation: ${target_allocation:.2f} USD")
                    print(f"📊 Current position: ${current_position:.2f} USD")

                    if current_position < target_allocation:
                        print(f"✨ Executing entry for {token}")
                        n.ai_entry(token, amount)
                        print(f"✅ Entry complete for {token}")
                    else:
                        print(f"⏸️ Position already at target size for {token}")

                except Exception as e:
                    print(f"❌ Error executing entry for {token}: {str(e)}")

                time.sleep(2)  # Small delay between entries

        except Exception as e:
            print(f"❌ Error executing allocations: {str(e)}")
            print("🔧 Moon Dev suggests checking the logs and trying again!")

    def handle_exits(self):
        """Check and exit positions based on SELL or NOTHING recommendations"""
        cprint("\n🔄 Checking for positions to exit...", "white", "on_blue")

        for _, row in self.recommendations_df.iterrows():
            token = row["token"]

            # Skip excluded tokens (USDC and SOL)
            if token in config.EXCLUDED_TOKENS:
                continue

            action = row["action"]

            # Check if we have a position
            current_position = n.get_token_balance_usd(token)

            if current_position > 0 and action in ["SELL", "NOTHING"]:
                cprint(
                    f"\n🚫 AI Agent recommends {action} for {token}",
                    "white",
                    "on_yellow",
                )
                cprint(
                    f"💰 Current position: ${current_position:.2f}", "white", "on_blue"
                )
                try:
                    cprint(
                        f"📉 Closing position with chunk_kill...", "white", "on_cyan"
                    )
                    n.chunk_kill(token, config.max_usd_order_size, config.slippage)
                    cprint(f"✅ Successfully closed position", "white", "on_green")
                except Exception as e:
                    cprint(f"❌ Error closing position: {str(e)}", "white", "on_red")
            elif current_position > 0:
                cprint(
                    f"✨ Keeping position for {token} (${current_position:.2f}) - AI recommends {action}",
                    "white",
                    "on_blue",
                )

    def parse_allocation_response(self, response):
        """Parse the AI's allocation response and handle both string and TextBlock formats"""
        try:
            # Handle TextBlock format from Claude 3
            if isinstance(response, list):
                response = (
                    response[0].text
                    if hasattr(response[0], "text")
                    else str(response[0])
                )

            print("🔍 Raw response received:")
            print(response)

            # Find the JSON block between curly braces
            start = response.find("{")
            end = response.rfind("}") + 1
            if start == -1 or end == 0:
                raise ValueError("No JSON object found in response")

            json_str = response[start:end]

            # More aggressive JSON cleaning
            json_str = (
                json_str.replace("\n", "")  # Remove newlines
                .replace("    ", "")  # Remove indentation
                .replace("\t", "")  # Remove tabs
                .replace("\\n", "")  # Remove escaped newlines
                .replace(" ", "")  # Remove all spaces
                .strip()
            )  # Remove leading/trailing whitespace

            print("\n🧹 Cleaned JSON string:")
            print(json_str)

            # Parse the cleaned JSON
            allocations = json.loads(json_str)

            print("\n📊 Parsed allocations:")
            for token, amount in allocations.items():
                print(f"  • {token}: ${amount}")

            # Validate amounts are numbers
            for token, amount in allocations.items():
                if not isinstance(amount, (int, float)):
                    raise ValueError(f"Invalid amount type for {token}: {type(amount)}")
                if amount < 0:
                    raise ValueError(f"Negative allocation for {token}: {amount}")

            return allocations

        except Exception as e:
            print(f"❌ Error parsing allocation response: {str(e)}")
            print("🔍 Raw response:")
            print(response)
            return None

    def parse_portfolio_allocation(self, allocation_text):
        """Parse portfolio allocation from text response"""
        try:
            # Clean up the response text
            cleaned_text = allocation_text.strip()
            if "```json" in cleaned_text:
                # Extract JSON from code block if present
                json_str = cleaned_text.split("```json")[1].split("```")[0]
            else:
                # Find the JSON object between curly braces
                start = cleaned_text.find("{")
                end = cleaned_text.rfind("}") + 1
                json_str = cleaned_text[start:end]

            # Parse the JSON
            allocations = json.loads(json_str)

            print("📊 Parsed allocations:")
            for token, amount in allocations.items():
                print(f"  • {token}: ${amount}")

            return allocations

        except json.JSONDecodeError as e:
            print(f"❌ Error parsing allocation JSON: {e}")
            print(f"🔍 Raw text received:\n{allocation_text}")
            return None
        except Exception as e:
            print(f"❌ Unexpected error parsing allocations: {e}")
            return None

    def run(self):
        """Run the trading agent (implements BaseAgent interface)"""
        self.run_trading_cycle()

    def run_trading_cycle(self, strategy_signals=None):
        """Run one complete trading cycle"""
        try:
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cprint(f"\n⏰ AI Agent Run Starting at {current_time}", "white", "on_green")

            # Collect OHLCV data for all tokens
            cprint("📊 Collecting market data...", "white", "on_blue")
            market_data = collect_all_tokens()

            # Analyze each token's data
            for token, data in market_data.items():
                cprint(f"\n🤖 AI Agent Analyzing Token: {token}", "white", "on_green")

                # Include strategy signals in analysis if available
                if strategy_signals and token in strategy_signals:
                    cprint(
                        f"📊 Including {len(strategy_signals[token])} strategy signals in analysis",
                        "cyan",
                    )
                    data["strategy_signals"] = strategy_signals[token]

                analysis = self.analyze_market_data(token, data)
                print(f"\n📈 Analysis for contract: {token}")
                print(analysis)
                print("\n" + "=" * 50 + "\n")

            # Show recommendations summary
            cprint("\n📊 Moon Dev's Trading Recommendations:", "white", "on_blue")
            summary_df = self.recommendations_df[
                ["token", "action", "confidence"]
            ].copy()
            print(summary_df.to_string(index=False))

            # Handle exits first
            self.handle_exits()

            # Then proceed with new allocations
            cprint(
                "\n💰 Calculating optimal portfolio allocation...", "white", "on_blue"
            )
            allocation = self.allocate_portfolio()

            if allocation:
                cprint("\n💼 Moon Dev's Portfolio Allocation:", "white", "on_blue")
                print(json.dumps(allocation, indent=4))

                cprint("\n🎯 Executing allocations...", "white", "on_blue")
                self.execute_allocations(allocation)
                cprint("\n✨ All allocations executed!", "white", "on_blue")
            else:
                cprint("\n⚠️ No allocations to execute!", "white", "on_yellow")

            # Clean up temp data
            cprint("\n🧹 Cleaning up temporary data...", "white", "on_blue")
            try:
                for file in os.listdir("temp_data"):
                    if file.endswith("_latest.csv"):
                        os.remove(os.path.join("temp_data", file))
                cprint("✨ Temp data cleaned successfully!", "white", "on_green")
            except Exception as e:
                cprint(f"⚠️ Error cleaning temp data: {str(e)}", "white", "on_yellow")

        except Exception as e:
            cprint(f"\n❌ Error in trading cycle: {str(e)}", "white", "on_red")
            cprint(
                "🔧 Moon Dev suggests checking the logs and trying again!",
                "white",
                "on_blue",
            )


def main():
    """Main function to run the trading agent every 15 minutes"""
    cprint("🌙 Moon Dev AI Trading System Starting Up! 🚀", "white", "on_blue")

    agent = TradingAgent()
    INTERVAL = config.SLEEP_BETWEEN_RUNS_MINUTES * 60  # Convert minutes to seconds

    while True:
        try:
            agent.run_trading_cycle()

            next_run = datetime.now() + timedelta(minutes=config.SLEEP_BETWEEN_RUNS_MINUTES)
            cprint(
                f"\n⏳ AI Agent run complete. Next run at {next_run.strftime('%Y-%m-%d %H:%M:%S')}",
                "white",
                "on_green",
            )

            # Sleep until next interval
            time.sleep(INTERVAL)

        except KeyboardInterrupt:
            cprint(
                "\n👋 Moon Dev AI Agent shutting down gracefully...", "white", "on_blue"
            )
            break
        except Exception as e:
            cprint(f"\n❌ Error: {str(e)}", "white", "on_red")
            cprint(
                "🔧 Moon Dev suggests checking the logs and trying again!",
                "white",
                "on_blue",
            )
            # Still sleep and continue on error
            time.sleep(INTERVAL)


if __name__ == "__main__":
    main()


In [2]:
agent = TradingAgent()

🤖 Moon Dev's LLM Trading Agent initialized!


In [ ]:
agent.run_trading_cycle().get_data()

In [44]:
import requests

COINGECKO_BASE_URL = "https://api.coingecko.com/api/v3"

# Fetch historical market data using CoinGecko API
def get_data(contract_address, days_back, timeframe):
    """
    Fetch historical market data for a Solana SPL token using CoinGecko API.
    
    Args:
        contract_address (str): The token's contract address.
        days_back (int): Number of days of historical data to fetch.
        timeframe (str): Timeframe for the data ("1h" for hourly, "1d" for daily).
    
    Returns:
        pd.DataFrame: A DataFrame containing historical price data.
    """
    # Map timeframe to CoinGecko's interval
    interval_map = {"1h": "hourly", "1d": "daily"}
    interval = interval_map.get(timeframe, "daily")

    # Construct the URL for historical market data
    url = f"{COINGECKO_BASE_URL}/coins/solana/contract/{contract_address}/market_chart?vs_currency=usd&days={days_back}&interval={interval}"
    headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": "CG-1ngQzwMyki64dw9xpMvqu694"
    }

    
    # Fetch historical market data
    response = requests.get(url=url, headers=headers)
    
    if response.status_code == 200:
        market_data = response.json()
        prices = market_data.get("prices", [])
        df = pd.DataFrame(prices, columns=["timestamp", "price"])
        df["Datetime (UTC)"] = pd.to_datetime(df["timestamp"], unit="ms")
        df = df.drop("timestamp", axis=1)
        return df
    else:
        print(
            f"❌ Failed to fetch market data for {contract_address}. Status code: {response.status_code}"
        )
        return pd.DataFrame()

In [51]:
from src.config import *

for token in MONITORED_TOKENS:
    days_back = DAYSBACK_4_DATA
    timeframe = "daily"
    result = get_data(contract_address=token, days_back=days_back, timeframe=timeframe)
    print(result)

      price      Datetime (UTC)
0  1.390212 2025-01-24 00:00:00
1  1.236401 2025-01-25 00:00:00
2  1.328935 2025-01-26 00:00:00
3  1.359706 2025-01-26 03:31:38
      price      Datetime (UTC)
0  0.897073 2025-01-24 00:00:00
1  0.807652 2025-01-25 00:00:00
2  0.820227 2025-01-26 00:00:00
3  0.845234 2025-01-26 03:31:42
      price      Datetime (UTC)
0  0.495911 2025-01-24 00:00:00
1  0.426514 2025-01-25 00:00:00
2  0.429223 2025-01-26 00:00:00
3  0.435204 2025-01-26 03:31:39
      price      Datetime (UTC)
0  1.419192 2025-01-24 00:00:00
1  1.275874 2025-01-25 00:00:00
2  1.281835 2025-01-26 00:00:00
3  1.284525 2025-01-26 03:31:46


In [25]:
headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": "CG-1ngQzwMyki64dw9xpMvqu694"
    }

test_url = f"{COINGECKO_BASE_URL}/ping"
response = requests.get(test_url, headers=headers)
print(response.json())

{'gecko_says': '(V3) To the Moon!'}


In [43]:
from src.config import *

for contract_address in MONITORED_TOKENS:
    days_back=4
    interval="daily"
    url = f"{COINGECKO_BASE_URL}/coins/solana/contract/{contract_address}/market_chart?vs_currency=usd&days={days_back}&interval={interval}"


    response = requests.get(url=url, headers=headers)
    print(f"Status Code: {response.status_code}")
    print(f"Response: {response.text}")

Status Code: 200
Response: {"prices":[[1737590400000,1.508923901366911],[1737676800000,1.3902119593487987],[1737763200000,1.2364010783490023],[1737849600000,1.328935229001619],[1737862071000,1.3579753106581076]],"market_caps":[[1737590400000,1529758397.980286],[1737676800000,1391061283.3498194],[1737763200000,1239134360.4439228],[1737849600000,1323852554.707728],[1737862071000,1359386680.0240173]],"total_volumes":[[1737590400000,459493812.67651093],[1737676800000,664101432.6653278],[1737763200000,256581843.62315556],[1737849600000,233721773.55937085],[1737862071000,233931835.37284967]]}
Status Code: 200
Response: {"prices":[[1737590400000,1.0080464833050833],[1737676800000,0.897072568415858],[1737763200000,0.8076515128250337],[1737849600000,0.8202267002021593],[1737862081000,0.8435451744288864]],"market_caps":[[1737590400000,1110172769.991984],[1737676800000,990448836.8129624],[1737763200000,886367456.3619473],[1737849600000,903551003.3437073],[1737862081000,929068073.5265267]],"total_

In [52]:
from src.config import *

def collect_token_data(contract_address, days_back=DAYSBACK_4_DATA, timeframe=DATA_TIMEFRAME):
    """Collect OHLCV data for a single token"""
    cprint(f"\n🤖 Moon Dev's AI Agent fetching data for {contract_address}...", "white", "on_blue")
    
    try:
        # Get data from CoinGecko
        data = n.get_data(contract_address, days_back, timeframe)
        
        if data is None or data.empty:
            cprint(f"❌ Moon Dev's AI Agent couldn't fetch data for {contract_address}", "white", "on_red")
            return None
            
        cprint(f"📊 Moon Dev's AI Agent processed {len(data)} candles for analysis", "white", "on_blue")
        
        # Save data if configured
        if SAVE_OHLCV_DATA:
            save_path = f"data/{contract_address}_latest.csv"
        else:
            save_path = f"temp_data/{contract_address}_latest.csv"
            
        # Ensure directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        # Save to CSV
        data.to_csv(save_path)
        cprint(f"💾 Moon Dev's AI Agent cached data for {contract_address[:4]}", "white", "on_green")
        
        return data
        
    except Exception as e:
        cprint(f"❌ Moon Dev's AI Agent encountered an error: {str(e)}", "white", "on_red")
        return None

In [53]:
for contract_address in MONITORED_TOKENS:
    data = collect_token_data(contract_address=contract_address)
    print(data)


🤖 Moon Dev's AI Agent fetching data for 9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump...
❌ Failed to fetch data for 9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump. Status code: 404
❌ Moon Dev's AI Agent couldn't fetch data for 9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump
None

🤖 Moon Dev's AI Agent fetching data for HeLp6NuQkmYB4pYWo2zYs22mESHXPQYzXbB8n4V98jwC...
❌ Failed to fetch data for HeLp6NuQkmYB4pYWo2zYs22mESHXPQYzXbB8n4V98jwC. Status code: 404
❌ Moon Dev's AI Agent couldn't fetch data for HeLp6NuQkmYB4pYWo2zYs22mESHXPQYzXbB8n4V98jwC
None

🤖 Moon Dev's AI Agent fetching data for KENJSUYLASHUMfHyy5o4Hp2FdNqZg1AsUPhfH2kYvEP...
❌ Failed to fetch data for KENJSUYLASHUMfHyy5o4Hp2FdNqZg1AsUPhfH2kYvEP. Status code: 404
❌ Moon Dev's AI Agent couldn't fetch data for KENJSUYLASHUMfHyy5o4Hp2FdNqZg1AsUPhfH2kYvEP
None

🤖 Moon Dev's AI Agent fetching data for EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm...
❌ Failed to fetch data for EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm. Status code: 404

In [37]:
def collect_all_tokens():
    """Collect OHLCV data for all monitored tokens"""
    market_data = {}
    
    cprint("\n🔍 Moon Dev's AI Agent starting market data collection...", "white", "on_blue")
    
    for contract_address in MONITORED_TOKENS:
        data = collect_token_data(contract_address)
        if data is not None:
            market_data[contract_address] = data
            time.sleep(6)  # Add a delay to avoid rate limiting
            
    cprint("\n✨ Moon Dev's AI Agent completed market data collection!", "white", "on_green")
    
    return market_data

In [39]:
market_data = collect_all_tokens()


🔍 Moon Dev's AI Agent starting market data collection...

🤖 Moon Dev's AI Agent fetching data for 9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump...
❌ Failed to fetch data for 9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump. Status code: 404
❌ Moon Dev's AI Agent couldn't fetch data for 9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump

🤖 Moon Dev's AI Agent fetching data for HeLp6NuQkmYB4pYWo2zYs22mESHXPQYzXbB8n4V98jwC...
❌ Failed to fetch data for HeLp6NuQkmYB4pYWo2zYs22mESHXPQYzXbB8n4V98jwC. Status code: 404
❌ Moon Dev's AI Agent couldn't fetch data for HeLp6NuQkmYB4pYWo2zYs22mESHXPQYzXbB8n4V98jwC

🤖 Moon Dev's AI Agent fetching data for KENJSUYLASHUMfHyy5o4Hp2FdNqZg1AsUPhfH2kYvEP...
❌ Failed to fetch data for KENJSUYLASHUMfHyy5o4Hp2FdNqZg1AsUPhfH2kYvEP. Status code: 404
❌ Moon Dev's AI Agent couldn't fetch data for KENJSUYLASHUMfHyy5o4Hp2FdNqZg1AsUPhfH2kYvEP

🤖 Moon Dev's AI Agent fetching data for EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm...
❌ Failed to fetch data for EKpQGSJtjMFqKZ9KQan

In [55]:
from src import nice_funcs as n

# Test get_data function
contract_address = "9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump"
days_back = 4
timeframe = "daily"

data = n.get_data(contract_address, days_back, timeframe)
print(data)

❌ Failed to fetch data for 9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump. Status code: 404
Empty DataFrame
Columns: []
Index: []
